## Test identify

In [1]:
(require '[whittemore.core :refer [define model q data identify]]
         '[clojure.pprint :refer [pprint]]
         :reload)

In [2]:
(define kidney
  "Model from Charig et. al. 1986"
  (model
    {:size []
     :treatment [:size]
     :recovery [:treatment :size]}))

In [3]:
(identify
    kidney
    (q [:recovery] :do [:treatment]))

Expected:

$$ \sum_\text{size} P(\text{size}) P(\text{recovery} \mid \text{size}, \text{treatment})$$

$$ \sum_{\text{size}} P(\text{recovery} \mid \text{size}, \text{treatment}) P(\text{size})$$

In [4]:
(define blood-pressure
  (model
    {:recovery [:bp :treatment]
     :bp [:treatment]
     :treatment []}))

In [5]:
(identify
  blood-pressure
  (q [:recovery] :do [:treatment]))

Expected:

$$ \sum_{bp} P(\text{bp} \mid \text{treatment}) P(\text{recovery} \mid \text{bp}, \text{treatment})  $$

$$ \sum_{bp} P(\text{recovery} \mid \text{bp}, \text{treatment}) P(\text{bp} \mid \text{treatment}) \frac{P(\text{treatment})}{P(\text{treatment})}$$

$$ \sum_{bp} P(\text{recovery}, \text{bp} \mid \text{treatment}) $$

$$ P(\text{recovery} \mid \text{treatment}) $$


In [6]:
(define wainer
  "See Causality, page 66"
  (model
    {:z_0 []
     :b [:z_0]
     :z_1 [:z_0]
     :x [:z_0]
     :z_2 [:z_1 :x]
     :z_3 [:z_2 :b]
     :y [:x :z_2 :z_3]}))

In [7]:
(identify
  wainer
  (data [:x :y :z_1 :z_2 :z_3])
  (q [:y] :do [:x]))

Expected:

$$
  \sum_{z_1, z_2, z_3}
  \left[ \sum_x P(z_1 \mid x) P(z_3 \mid x_1, z_1, z_2) P(x) \right]
  P(z_2 \mid x, z_1) P(y \mid x, z_1, z_2, z_3) 
$$

$$
  \sum_{z_1, z_2, z_3}
  P(z_2 \mid x, z_1) P(y \mid x, z_1, z_2, z_3)
  \sum_x P(z_1 \mid x) P(z_3 \mid x_1, z_1, z_2) P(x)
$$

$$
  \sum_{z_1, z_2, z_3}
  P(y \mid x, z_1, z_2, z_3) P(z_2 \mid x, z_1)
  \sum_x P(z_3 \mid x_1, z_1, z_2) P(z_1 \mid x) P(x)
$$

Note that $y \perp\!\!\perp z_1 \mid x, z_2, z_3$, therefore $P(y \mid x, z_1, z_2, z_3) = P(y \mid z_2, z_3, x)$

$$
  \sum_{z_1} \sum_{z_2} \sum_{z_3}
  P(y \mid z_2, z_3, x) P(z_2 \mid z_1, x)
  \times \sum_{x'} P(z_3 \mid z_1, z_2, x') P(z_1, x')
$$

In [8]:
(define causality-f
  "Causality, page 93"
  (model
    {:y [:z_1 :z_2 :x]
     :z_2 [:z_1]
     :z_1 [:x]
     :x []}
    #{:y :z_1}
    #{:x :z_2}))

In [9]:
(identify
  causality-f
  (q [:y] :do [:x]))

Expected:

$$ \sum_{z_1, z_2} \left[ \sum_x P(x) P(z_2 \mid x, z_1) \right] P(z_1 \mid x) P(y \mid x, z_1, z_2)$$

$$ \sum_{z_1, z_2} P(z_1 \mid x) P(y \mid x, z_1, z_2) \sum_x P(x) P(z_2 \mid x, z_1) $$

$$ \sum_{z_1, z_2} P(y \mid z_1, z_2, x) P(z_1 \mid x) \sum_{x'} P(z_2 \mid z_1, x') P(x') $$

In [10]:
(define causality-g
  "Causality, page 93"
  (model
    {:z_2 []
     :x [:z_2]
     :z_1 [:x :z_2]
     :z_3 [:z_2]
     :y [:z_1 :z_3]}
    #{:x :z_2}
    #{:y :z_2}
    #{:x :y}
    #{:x :z_3}))

In [11]:
(identify
    causality-g
    (q [:y] :do [:x]))

Equals:

$$ \sum_{z_1, z_2, z_3} \frac{\sum_{z_1, x} P(z_3 | x, z_2) P(z_2) P(x | z_2) P(y | x, z_1, z_2, z_3)}{\sum_y ...} $$
$$ \times \frac{\sum_{y, z_1, z_3} \sum_x P(z_3 | x, z_2) P(z_2) P(x | z_2) P(y | x, z_1, z_2, z_3)}{\sum_{z_2} ...} $$
$$ \times P(z_1 | x, z_2) P(z_3 | z_2)$$

$$ \sum_{z_1, z_2, z_3} \frac{ \sum_{z_1, x} P(y | x, z_1, z_2, z_3) P(x, z_2, z_3) }{ \sum_y ...}$$
$$ \times \frac{ \sum_{x, y, z_1, z_3} P(y | x, z_1, z_2, z_3) P(x, z_2, z_3) }{ \sum_{z_2 ...}}$$
$$ \times P(z_1 | x, z_2) P(z_3 | z_2) $$

Note: $ x \perp\!\!\!\perp z_2 | z_1, z_3 $

Fully simplified:

$$ \sum_{z_1} \sum_{z_2} \sum_{x'} P(y \mid z_1, z_2, x') P(x' \mid z_2) \times P(z_1 \mid z_2, x) P(z_2) $$

In [12]:
(pprint
  (identify
    causality-g
    (q [:y] :do [:x])))

{:form
 {:sub #{:z_2 :z_1 :z_3},
  :sum
  {:prod
   #{{:prod
      #{{:numer
         {:sub #{:z_1},
          :sum
          {:sub #{:x},
           :sum
           {:prod
            #{{:numer
               {:sub #{:y :z_1}, :sum {:p #{:y :z_2 :z_1 :x :z_3}}},
               :denom
               {:sub #{:z_3},
                :sum
                {:sub #{:y :z_1}, :sum {:p #{:y :z_2 :z_1 :x :z_3}}}}}
              {:numer
               {:sub #{:y :z_1 :x :z_3},
                :sum {:p #{:y :z_2 :z_1 :x :z_3}}},
               :denom
               {:sub #{:z_2},
                :sum
                {:sub #{:y :z_1 :x :z_3},
                 :sum {:p #{:y :z_2 :z_1 :x :z_3}}}}}
              {:numer
               {:sub #{:y :z_1 :z_3},
                :sum {:p #{:y :z_2 :z_1 :x :z_3}}},
               :denom
               {:sub #{:x},
                :sum
                {:sub #{:y :z_1 :z_3},
                 :sum {:p #{:y :z_2 :z_1 :x :z_3}}}}}
              {:numer {:p #{:y :

Should throw exception (args out of order):

In [13]:
(identify
  wainer
  (q [:y] :do [:x])
  (data [:x :y]))

ExceptionInfo Unsupported Data type (check args)  clojure.core/ex-info (core.clj:4617)


class clojure.lang.ExceptionInfo: 